In [1]:
import os, sys
import numpy as np
import pandas as pd
import pickle

curr_dir = sys.path[0]
pretrained_dir = os.path.join(curr_dir, '../AM2019/pretrained/Jan26-icml/')
baseline_dir = os.path.join(curr_dir, '../PyConcorde')
data_dir = "/home/ecal_team/datasets/amazon_vrp/amazon_processed"

for dir in [pretrained_dir, baseline_dir, data_dir]:
    assert os.path.exists(dir), f"Directory {dir} does not exist"

In [2]:
def get_dataset_fn(ds, n, problem="tsp"):
    if ds == 'amz_nS' or ds == 'amz_S':
        assert problem == 'tsp', "amzaon dataset only support TSP now"
        data_fn = f"amazon_eval_N{n}_I1000_seed1234_S.pkl"
    elif ds == 'rnd_S' or ds == 'rnd_C':
        if problem == 'cvrp':
            assert ds == 'rnd_S', "rnd complex dataset only support TSP now"
        data_fn = f"{'CVRP_'*(problem=='cvrp')}rnd_N{n}_I1000_{ds[-1]}_seed1234_iter4.pkl"
    elif ds == 'amz_eval':
        raise NotImplementedError
    return data_fn


def load_pkl_file(fn):
    if os.path.exists(fn):
        with open(fn, 'rb') as f:
            data = pickle.load(f)
    else:
        print(f"File {fn.split('/')[-1]} does not exist")
        data = None
    return data

def load_baselines(problem = "tsp"):
    if problem == "cvrp":
        methods = ["lkh"]
        ds_list = ["rnd_S"]
    else:
        methods = ["concorde", "lkh"]
        ds_list = ["amz_nS", "amz_S", "rnd_S", "rnd_C"]
    prefix = {"concorde": "CCD", "lkh": "LKH"}
    baselines = {k: dict() for k in methods}
    for method in methods:
        for ds in ds_list:
            baselines[method][ds] = dict()
            for n in [20, 50, 100]:
                ds_fn = get_dataset_fn(ds, n, problem=problem)
                is_amz_nS = (ds == "amz_nS")
                fn = f"{prefix[method]}_{'nS_'*is_amz_nS}{ds_fn}".replace("NoTrack", "")
                baselines[method][ds][n] = load_pkl_file(os.path.join(baseline_dir, fn))
    return baselines

tsp_baselines = load_baselines(problem="tsp")
cvrp_baselines = load_baselines(problem="cvrp")
baselines = {
    "tsp": tsp_baselines,
    "cvrp": cvrp_baselines
}

In [3]:
def get_model_eval(model, ds, width=None):
    default_width = {"greedy": 0, "sample": 1000, "bs": 1000}
    width = dict() if width is None else width
    default_width.update(width)

    model_dir = os.path.join(pretrained_dir, model)
    assert os.path.exists(model_dir), f"Model directory {model_dir} does not exist"
    data = {
        s: load_pkl_file(os.path.join(model_dir, f"Res_D_{ds}_{s}-{default_width[s]}.pkl"))
        for s in default_width.keys()
    }
    for s in list(data.keys()):
        if data[s] is None:
            del data[s]
    return data


In [4]:
def compare_single(model_data, ref_data):
    model_obj = model_data["obj"]
    ref_obj = ref_data["obj"]
    
    valid_idx = np.where((ref_obj > 0) * (ref_obj < np.inf) *\
                         (model_obj > 0) * (model_obj < np.inf) == 1)[0]
    if len(valid_idx) < len(model_obj):
        print(f"valid idx: {len(valid_idx)} out of {len(model_obj)}")
    model_obj = model_obj[valid_idx]
    ref_obj = ref_obj[valid_idx]

    model_obj_mean = np.mean(model_obj)
    opt_gap = (model_obj - ref_obj) / ref_obj
    opt_gap_mean = np.mean(opt_gap)
    opt_gap_5 = np.percentile(opt_gap, 5)
    opt_gap_95 = np.percentile(opt_gap, 95)

    model_time = model_data["time"][valid_idx].sum()

    # significance test

    return {
        "Obj": model_obj_mean,
        "Gap": opt_gap_mean,
        "Time": model_time,
        "Gap (5%)": opt_gap_5,
        "Gap (95%)": opt_gap_95,
        
    }


def compare_performance(model, ref, ds, n=None, problem="tsp"):
    if model in ["concorde", "lkh"]:
        s = "exact" if model == "concorde" else "meta-H"
        model_data = {s: baselines[problem][model][ds][n]}
    else:
        model_data = get_model_eval(model, ds)
        n = n if n is not None else int(model.split("_")[2].split(problem)[-1])
    ref_data = baselines[problem][ref][ds][n]
    
    res_table = {k: dict() for k in model_data.keys()}
    for s in model_data.keys():
        res = compare_single(model_data[s], ref_data)
        if model not in ["concorde", "lkh"]:
            time_div = 1000 if s == "greedy" else 10 # FIXME
            res["Time"] /= time_div
        res_table[s] = res

    return res_table

    


In [5]:
# format time: xxx (s) -> xx H xx M xx S
def format_time(t):
    t_string = ""
    h = int(t // 3600)
    t_string += f"{h}H " if h > 0 else ""
    m = int((t % 3600) // 60)
    t_string += f"{m}M " if m > 0 else ""
    s = t % 60
    t_string += f"{s:.2f}S" if t_string == "" else f"{s:.0f}S"
    return t_string

def format_time_2(t):
    remove_float = lambda x: f"{x:.0f}" if x > 5 else f"{x:.1f}"
    if t > 86400:
        d = remove_float(t / 86400)
        t_string = f"{d} d"
    elif t > 3600:
        h = remove_float(t / 3600)
        t_string = f"{h} h"
    elif t > 60:
        m = remove_float(t / 60)
        t_string = f"{m} m"
    else:
        s = remove_float(t)
        t_string = f"{s} s"
    return t_string

In [156]:
def latex_lower_decimal(number, digit=2, pct=False):
    # Split the number into integer and decimal parts
    if pct:
        number *= 100
    integer_part, decimal_part = str(number).split('.')

    # Format the LaTeX string'
    pct_notation = "\\%" if pct else ""
    latex_string = f"{integer_part}_{{.{decimal_part[:digit]}{pct_notation}}}"

    return latex_string

12_{.34\%}


In [6]:
def format_model_res(res_dict, drop_pct=False, drop_time=False, latex_format=False):
    df = pd.DataFrame.from_dict(res_dict, orient="index", columns=["Obj", "Gap", "Time", "Gap (5%)", "Gap (95%)"])
    # Obj: .2f, Gap: .1%, Time: .3f, Gap (5%): .1%, Gap (95%): .1%
    if not latex_format:
        df["Obj"] = df["Obj"].apply(lambda x: f"{x:.2f}")
        df["Gap"] = df["Gap"].apply(lambda x: f"{x*100:.1f}%")
        df["Time"] = df["Time"].apply(lambda x: format_time_2(x))
        df["Gap (5%)"] = df["Gap (5%)"].apply(lambda x: f"{x*100:.1f}%")
        df["Gap (95%)"] = df["Gap (95%)"].apply(lambda x: f"{x*100:.1f}%")
    else:
        # df["Obj"] = df["Obj"].apply(lambda x: latex_lower_decimal(x, digit=2))
        # df["Gap"] = df["Gap"].apply(lambda x: latex_lower_decimal(x, digit=1, pct=True))
        df["Obj"] = df["Obj"].apply(lambda x: f"{x:.2f}")
        df["Gap"] = df["Gap"].apply(lambda x: f"{x*100:.1f}\%")
        df["Time"] = df["Time"].apply(lambda x: f"({format_time_2(x)})")
        df["Gap (5%)"] = df["Gap (5%)"].apply(lambda x: f"{x*100:.1f}")
        df["Gap (95%)"] = df["Gap (95%)"].apply(lambda x: f"{x*100:.1f}")
    if drop_pct:
        df = df.drop(columns=["Gap (5%)", "Gap (95%)"])
    if drop_time:
        df = df.drop(columns=["Time"])
    return df

In [7]:
def generate_Table_1(latex_format=False):
    to_concat = []
    for n in [20, 50, 100]:
        ds = "rnd_S"
        models_Table_1 = {
            "Concorde": "concorde",
            "LKH-3": "lkh",
            "AM": f"icml_nE_tsp{n}_ref",
            "AM-N": f"icml_nE_tsp{n}_relK5",
            "AM-E": f"icml_nE_tsp{n}_orgE3",
            "AM-nE*": f"icml_nE_tsp{n}_relK5_orgE6_ultra",
        }

        df = pd.concat([format_model_res(
            compare_performance(models_Table_1[k], "concorde", ds, n=n),
            drop_pct=True, latex_format=latex_format) for k in models_Table_1.keys()], 
            axis=0, keys=models_Table_1.keys())
        to_concat.append(df)
    return pd.concat(to_concat, axis=1, keys=[20, 50, 100]).replace(pd.NA, "")

df1 = generate_Table_1(latex_format=True)
df1.to_csv("Table_1.csv")
df1

20                      50                      100          \
                  Obj     Gap     Time    Obj     Gap     Time    Obj     Gap   
Concorde exact   4.67   0.0\%  (1.8 m)   8.24   0.0\%   (24 m)  12.55   0.0\%   
LKH-3    meta-H  4.67   0.0\%  (4.4 m)   8.24   0.0\%   (28 m)  12.55   0.0\%   
AM       greedy  5.68  21.5\%  (0.2 s)  10.40  26.1\%   (11 s)  16.13  28.5\%   
         sample  5.48  17.3\%  (1.3 m)   9.88  19.8\%   (27 m)  15.34  22.2\%   
         bs      5.21  11.4\%  (1.5 m)   9.86  19.6\%    (9 m)  15.55  23.9\%   
AM-N     greedy  5.20  11.3\%  (0.2 s)   9.86  19.6\%   (32 s)  15.77  25.6\%   
         sample  5.03   7.6\%  (1.3 m)   9.38  13.7\%   (27 m)  15.01  19.5\%   
         bs      4.88   4.3\%  (1.5 m)   9.40  14.0\%    (9 m)  15.24  21.4\%   
AM-E     greedy  4.91   4.9\%  (0.2 s)   8.96   8.7\%  (4.0 s)  14.23  13.3\%   
         sample  4.80   2.6\%  (1.3 m)   8.65   5.0\%    (7 m)  13.69   9.0\%   
         bs      4.73   1.3\%  (1.5 m)   8.66   5.0\%    (9 m)  13.81  10.0\%   
AM-nE*   greedy  4.84   3.5\%    (5 s)   8.73   5.9\%    (8 s)  13.66   8.8\%   
         sample  4.74   1.4\%    (9 m)   8.48   2.9\%    (7 m)  13.14   4.7\%   
         bs      4.71   0.8\%   (13 m)   8.48   2.8\%    (9 m)  13.14   4.7\%   

                          
                    Time  
Concorde exact   (3.9 h)  
LKH-3    meta-H  (1.4 h)  
AM       greedy  (2.6 s)  
         sample   (35 m)  
         bs       (43 m)  
AM-N     greedy  (2.7 s)  
         sample   (32 m)  
         bs       (45 m)  
AM-E     greedy  (3.8 s)  
         sample   (34 m)  
         bs       (44 m)  
AM-nE*   greedy    (8 s)  
         sample   (31 m)  
         bs       (43 m)

In [8]:
def generate_Table_2(latex_format=False):
    to_concat = []
    for n in [20, 50, 100]:
        ds_to_concat = []
        for ds in ["rnd_C", "amz_nS"]:    
            models_Table_1 = {
                "Concorde": "concorde",
                "LKH-3": "lkh",
                "AM": f"icml_nE_tsp{n}_ref",
                # "AM-N": f"icml_nE_tsp{n}_relK5",
                # "AM-E": f"icml_nE_tsp{n}_orgE3",
                "AM-nE*": f"icml_nE_tsp{n}_relK5_orgE6_ultra",
            }

            df = pd.concat([format_model_res(
                compare_performance(models_Table_1[k], "concorde", ds, n=n),
                drop_pct=True, drop_time=True, latex_format=latex_format) for k in models_Table_1.keys()], 
                axis=0, keys=models_Table_1.keys())
            ds_to_concat.append(df)
        to_concat.append(pd.concat(ds_to_concat, axis=0, keys=["rnd_C", "amz_nS"]))
    # return to_concat
    return pd.concat(to_concat, axis=1, keys=[20, 50, 100]).replace(pd.NA, "")

df2 = generate_Table_2(latex_format=True)
df2.to_csv("Table_2.csv")
df2

valid idx: 999 out of 1000
valid idx: 999 out of 1000
valid idx: 999 out of 1000
valid idx: 999 out of 1000
valid idx: 999 out of 1000
valid idx: 999 out of 1000
valid idx: 999 out of 1000
valid idx: 999 out of 1000
valid idx: 996 out of 1000
valid idx: 996 out of 1000
valid idx: 996 out of 1000
valid idx: 996 out of 1000
valid idx: 996 out of 1000
valid idx: 996 out of 1000
valid idx: 996 out of 1000
valid idx: 996 out of 1000


20               50                100        
                            Obj     Gap      Obj     Gap       Obj     Gap
rnd_C  Concorde exact      4.03   0.0\%     7.02   0.0\%     10.61   0.0\%
       LKH-3    meta-H     4.03   0.0\%     7.02   0.0\%     10.61   0.0\%
       AM       greedy     4.87  21.0\%     8.90  27.1\%     13.85  30.9\%
                sample     4.69  16.3\%     8.40  19.6\%     13.33  26.7\%
                bs         4.48  11.2\%     8.42  20.0\%     13.41  27.1\%
       AM-nE*   greedy     4.14   2.8\%     7.45   6.2\%     11.55   9.1\%
                sample     4.07   0.9\%     7.22   2.8\%     11.07   4.5\%
                bs         4.05   0.4\%     7.22   2.8\%     11.12   5.0\%
amz_nS Concorde exact   2802.56   0.0\%  4450.45   0.0\%   6445.34   0.0\%
       LKH-3    meta-H  2802.57   0.0\%  4450.47   0.0\%   6445.45   0.0\%
       AM       greedy  3136.23  12.9\%  5888.62  33.5\%   9748.78  52.3\%
                sample  2993.24   7.5\%  5324.01  20.4\%  10100.96  56.3\%
                bs      2934.55   5.2\%  5450.45  23.1\%  10032.59  55.4\%
       AM-nE*   greedy  2952.12   5.9\%  5073.77  15.4\%   8402.89  31.6\%
                sample  2851.42   1.9\%  4805.20   8.8\%   7905.65  23.0\%
                bs      2836.91   1.3\%  4843.27   9.6\%   8092.73  26.2\%

In [9]:
def generate_Table_3(latex_format=False):
    to_concat = []
    problem = "cvrp"
    for n in [20, 50, 100]:
        ds = "rnd_S"
        models_Table_1 = {
            # "Concorde": "concorde",
            "LKH-3": "lkh",
            "AM": f"icml_nE_{problem}{n}_ref",
            "AM-nE*": f"icml_nE_{problem}{n}_relK5_orgE3",
        }

        df = pd.concat([format_model_res(
            compare_performance(models_Table_1[k], "lkh", problem="cvrp", ds=ds, n=n),
            drop_pct=True, latex_format=latex_format) for k in models_Table_1.keys()], 
            axis=0, keys=models_Table_1.keys())
        to_concat.append(df)
    return pd.concat(to_concat, axis=1, keys=[20, 50, 100]).replace(pd.NA, "")

df3 = generate_Table_3(latex_format=True)
df3.to_csv("Table_3.csv")
df3

valid idx: 191 out of 1000
valid idx: 191 out of 1000
valid idx: 191 out of 1000
valid idx: 191 out of 1000
valid idx: 191 out of 1000
valid idx: 191 out of 1000
valid idx: 191 out of 1000
valid idx: 166 out of 1000
valid idx: 166 out of 1000
valid idx: 166 out of 1000
valid idx: 166 out of 1000
valid idx: 166 out of 1000
valid idx: 166 out of 1000
valid idx: 166 out of 1000


20                      50                      100          \
                Obj     Gap     Time    Obj     Gap     Time    Obj     Gap   
LKH-3  meta-H  6.80   0.0\%    (7 h)  12.49   0.0\%  (1.2 d)  19.43   0.0\%   
AM     greedy  8.15  19.9\%  (0.2 s)  15.44  23.8\%  (0.1 s)  24.42  25.7\%   
       sample  7.63  12.3\%  (2.0 m)  14.59  16.9\%  (1.6 m)  23.14  19.1\%   
       bs      7.43   9.3\%  (2.0 m)  14.64  17.4\%  (1.9 m)  23.58  21.4\%   
AM-nE* greedy  7.21   5.9\%  (0.2 s)  13.82  10.7\%  (0.2 s)  22.01  13.3\%   
       sample  6.89   1.2\%  (1.7 m)  13.11   5.0\%  (1.6 m)  21.05   8.3\%   
       bs      6.81   0.1\%  (2.0 m)  13.19   5.6\%  (2.0 m)  21.29   9.6\%   

                        
                  Time  
LKH-3  meta-H  (1.2 d)  
AM     greedy  (0.6 s)  
       sample    (6 m)  
       bs        (8 m)  
AM-nE* greedy  (0.7 s)  
       sample    (7 m)  
       bs        (9 m)

In [182]:
ds = "rnd_S"
problem = "cvrp"
n = 100
model = f"icml_nE_{problem}{n}_relK5_orgE3"
format_model_res(compare_performance(model, "lkh", problem="cvrp", ds=ds, n=n))

File Res_D_rnd_S_greedy-0.pkl does not exist
File Res_D_rnd_S_sample-1000.pkl does not exist
File Res_D_rnd_S_bs-1000.pkl does not exist


,Obj,Gap,Time,Gap (5%),Gap (95%)
